In [1]:
from hello.pid.lvpid import PIDController
from hello.pid.delay import *

In [2]:
from officelib.xllib import *
xl = Excel()

wb = xl.ActiveWorkbook
wb = wb or xl.Workbooks.Add()

ws = xl.ActiveSheet
ws = ws or wb.Worksheets.Add()

cells = ws.Cells
cell_range = ws.Cells.Range

In [8]:
from math import sin, pi
class AgProcess():
    def __init__(self, m=1/2.76, b=0, delay=3, op=0, pv=0, pamplitude=0.01, pperiod=30, ag_min=3):
        self.delay = DelayBuffer(delay, pv).cycle
        self.m = m
        self.b = b
        self.x = self.t = 0
        self.pamplitude = pamplitude
        self.pperiod = pperiod or 1
        self.pv = 0
        self.ag_min = ag_min
        
        
    def step(self, op):
        self.x += 1
        self.t += 1
        if self.x >= self.pperiod:
            self.x = 0
        m = self.m
        f = sin(self.x*2*pi/self.pperiod)*self.pamplitude + 1
        m *= f
        op = self.delay(op)
        self.pv = m*op + self.b
        if self.pv < self.ag_min:
            return 0
        return self.pv
    
    def step(self, op):
        self.pv = self.m * op + self.b
        return max(self.ag_min, self.delay(self.pv))

In [4]:
def ag_sim(p,i,d,end,delay=3, op=0,pv=0,sp=8, amp=0.05, per=180):
    pid = PIDController(p,(i),d,100,0)
    pid.man_to_auto(pv, sp, op)
    proc = AgProcess(delay=delay, pamplitude=amp, pperiod=per)
    t = 0
    data = [(t, pv, op)]
    while True:
        t += 1
        if t < 15:
            pv = 0
        op = pid.step(pv, sp)
        data.append((t, pv, op))
        pv = proc.step(op)
        if t >= end: break
    return data

In [5]:
def paste(cells, data):
    cells.Range(cells(1,1), cells(1, 3)).Value = [("T", "PV", "OP")]
    topleft = cells(2,1)
    bottomright = topleft.Offset(len(data), len(data[0]))
    cells.Range(topleft, bottomright).Clear()
    cells.Range(topleft, bottomright).Value = data

def clear(cells, data):
    topleft = cells(2,1)
    bottomright = topleft.Offset(len(data), len(data[0]))
    cells.Range(topleft, bottomright).Clear()
    

In [ ]:
clear(cells, data)
data = ag_sim(0.3, 0.02, 0, 180, 10, 0, 0, 8, 0.005*0, 50)
paste(cells, data)

In [ ]:
data = ag_sim(0.1, 0.01, 0, 180, 4, 0, 0, 12, 0.005, 50)
paste(cells, data)

In [24]:
%matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

Using matplotlib backend: TkAgg


In [27]:
def ag_sim2(p,i,d,end,delay=3, op=0,pv=0,sp=8, amp=0.05, per=180):
    global pid, proc
    pid = PIDController(p,i,d,100,0,0)
    pid.off_to_auto(pv, sp)
    proc = AgProcess(59/100, 0, delay=delay, pv=pv,pamplitude=0, pperiod=1)
    t = 0
    data = [(t, pv, op)]
    while True:
        op = pid.step(pv, sp)
        data.append((t, pv, op))
        pv = proc.step(op)
        if t >= end: break
        t += 1
    return data

data = ag_sim2(.1, 0.008, 0, 180, 4, 0, 0, 12)
t, pv, op = list(zip(*data))
ax1.clear()
ax2.clear()
ax1.scatter(t, pv, label="PV")
ax2.scatter(t, op, label="OP")
ax1.set_xlim(0, 180)

ax2.set_xlim(0, 180)

(0, 180)